In [37]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split



In [50]:
def initialize_parameters():
    initializer = tf.keras.initializers.GlorotUniform(seed=0)
    W1 = tf.Variable(initializer(shape=(4, 4, 3, 8)), name="W1", dtype=tf.float32)
    W2 = tf.Variable(initializer(shape=(2, 2, 8, 16)), name="W1", dtype=tf.float32)
    parameters = {}
    parameters["W1"] = W1
    parameters["W2"] = W2
    
    return parameters

In [40]:
W = initialize_parameters()
print(W)

(<tf.Variable 'W1:0' shape=(4, 4, 3, 8) dtype=float32, numpy=
array([[[[-1.06714614e-01,  7.60742277e-02,  1.10805735e-01,
           9.99966711e-02, -4.11413610e-02,  1.36764273e-01,
           1.13805756e-01, -1.11911990e-01],
         [ 1.28976747e-01,  1.48359835e-02, -1.34384513e-01,
          -1.43575072e-01,  9.12466496e-02, -1.11755095e-01,
          -5.25314212e-02,  7.49562681e-03],
         [-2.72752047e-02, -1.06626131e-01, -1.04283683e-01,
          -1.25989273e-01, -1.24091312e-01,  1.83669612e-01,
          -3.20947021e-02, -1.10677637e-01]],

        [[ 8.13676268e-02, -1.57865196e-01, -8.02577138e-02,
          -1.13867439e-01, -1.67241648e-01,  6.69556707e-02,
          -1.21889599e-01, -3.32291722e-02],
         [-1.76359713e-01,  8.11824054e-02, -9.66130346e-02,
           3.13896686e-02,  1.71372592e-02, -1.07769795e-01,
          -1.70043230e-01,  4.14290875e-02],
         [ 1.53008923e-01,  1.53686866e-01, -8.14601034e-02,
           1.81735203e-01,  1.50680527e-

In [ ]:
def forward_propagation(X, parameters):
    ''' 
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    '''
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X, W1, strides=1, padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool2d(A1, ksize=8, strides=8, padding='SAME')
    Z2 = tf.nn.conv2d(P1, W2, strides=1, padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool2d(A2, ksize=4, strides=4, padding='SAME')
    P = tf.keras.layers.Flatten()(P2)
    Z3 = tf.keras.layers.Dense(11, activation=None)(P)
    return Z3

In [42]:
def compute_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y))
    return cost

In [43]:
def random_mini_batches(X, Y, minibatch_size=64, seed=0):
    np.random.seed(seed)
    m = X.shape[0]
    mini_batches = []
    
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, ...]
    shuffled_Y = Y[permutation, ...]
    
    num_complete_minibatch = m // minibatch_size
    for k in range(num_complete_minibatch):
        mini_batch_X = shuffled_X[k * minibatch_size: (k + 1) * minibatch_size, ...]
        mini_batch_Y = shuffled_Y[k * minibatch_size: (k + 1) * minibatch_size, ...]
        mini_batches.append((mini_batch_X, mini_batch_Y))
    if m % minibatch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatch * minibatch_size:, ...]
        mini_batch_Y = shuffled_Y[num_complete_minibatch * minibatch_size:, ...]
        mini_batches.append((mini_batch_X, mini_batch_Y))
    
    return mini_batches
        

In [51]:
def model(X_train, Y_train, X_test, Y_test, learning_rate=0.009,num_epochs=100, minibatch_size=64, print_cost=True):
    tf.compat.v1.reset_default_graph()
    tf.random.set_seed(1)
    seed = 3
    np.random.seed(seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape
    n_y = Y_train.shape[1]
    costs = []
    
    parameters = initialize_parameters()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    for epoch in range(num_epochs):
        minibatch_cost = 0
        seed += 1
        minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed=seed)
        
        for (minibatch_X, minibatch_Y) in minibatches:
            with tf.GradientTape() as tape:
                Z3 = forward_propagation(minibatch_X, parameters)
                loss = compute_cost(Z3, minibatch_Y)
            trainable_variables = list(parameters.values())
            gradients = tape.gradient(loss, trainable_variables)
            optimizer.apply_gradients(zip(gradients, trainable_variables))
            minibatch_cost += loss.numpy()

        costs.append(minibatch_cost)

        if print_cost and epoch % 10 == 0:
            print(f"Cost after epoch {epoch}: {minibatch_cost}")
    
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    Z3_test = forward_propagation(X_test, parameters)
    predictions = tf.argmax(Z3_test, axis=1)  # Chọn lớp có xác suất cao nhất
    labels = tf.argmax(Y_test, axis=1)  # Nhãn thực tế

    # Tính số lượng dự đoán đúng
    correct_predictions = tf.reduce_sum(tf.cast(tf.equal(predictions, labels), dtype=tf.float32))

    # Tính accuracy
    accuracy = correct_predictions.numpy() / X_test.shape[0]
    
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    
    
    return parameters
            
    

In [45]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("muhammadkhalid/sign-language-for-numbers")

print("Path to dataset files:", path)

Path to dataset files: /Users/minhquang/.cache/kagglehub/datasets/muhammadkhalid/sign-language-for-numbers/versions/1


In [46]:
import os
dataset_path = "/Users/minhquang/.cache/kagglehub/datasets/muhammadkhalid/sign-language-for-numbers/versions/1"
print("Dataset files: ", os.listdir(dataset_path))

Dataset files:  ['Sign Language for Numbers']


In [47]:
dataset_dir = "/Users/minhquang/.cache/kagglehub/datasets/muhammadkhalid/sign-language-for-numbers/versions/1/Sign Language for Numbers"
print("Files in dataset directory:", os.listdir(dataset_dir))

Files in dataset directory: ['9', '0', '7', '6', '1', '8', 'unknown', '4', '3', '2', '5']


In [48]:
IMG_SIZE = 64
X, Y = [], []
labels = sorted(os.listdir(dataset_dir))
label_map = {label: idx for idx, label in enumerate(labels)}
for label in labels:
    class_dir = os.path.join(dataset_dir, label)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize ảnh về kích thước chuẩn
            X.append(img)
            Y.append(label_map[label])
            
X = np.array(X) / 255.0
Y = np.array(Y)
print(label_map)
print(len(label_map))  # Xem tổng số lớp

# Chuyển đổi nhãn thành One-Hot Encoding
num_classes = len(label_map)  
Y = tf.keras.utils.to_categorical(Y, num_classes=11)

# Chia dữ liệu thành train/test (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print(f"Train set: {X_train.shape}, {Y_train.shape}")
print(f"Test set: {X_test.shape}, {Y_test.shape}")

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'unknown': 10}
11
Train set: (13200, 64, 64, 3), (13200, 11)
Test set: (3300, 64, 64, 3), (3300, 11)


In [ ]:
model(X_train, Y_train, X_test, Y_test, learning_rate=0.009,num_epochs=100, minibatch_size=64, print_cost=True)